# Langchain
LangChain es una de las bibliotecas de código abierto más populares para ingenieros de IA. Su objetivo es simplificar la creación de software de IA, proporcionar bloques de construcción fáciles de usar y facilitar la transición entre proveedores de servicios de IA.

En este ejemplo, presentaremos LangChain, creando un asistente sencillo basado en LLM. Proporcionaremos ejemplos para llama3.2 de Meta a través de Ollama!

Para ello trabajaremos en un ambiente virtual con VENV lo llamaremos *"venvnb"*, instalaremos prerequisitos para iniciar con Ollama y Langchain.
Seguidamente estaremos trabajando sobre el prompt template

In [38]:
!python -m venv venvnb || source venvnb\Scripts\activate
!pip install -qU langchain-core langchain-ollama langchain-community

Inicializamos el model con llama3.2 local

In [39]:
from langchain_ollama.llms import OllamaLLM

# Inicializa el modelo Ollama LLM
model = OllamaLLM(model="llama3.2")

A continuación, estaremos implementando el system y el user prompt apoyándonos de templates de langchain. Estos nos permiten trabajar de forma dinámica dentro de cada caso estableciendo variables que nos ayudan a mejorar el contexto y a la vez ahorrar tokens desde el inicio.

In [40]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

empresa = "Tech Innovators"
word_list = ["Innovate", "Future", "Tech", "Solutions", "NextGen"]

# Definiendo el System Prompt
system_prompt = SystemMessagePromptTemplate.from_template(
    "Eres un asistente creativo que genera slogans de empresas basados en el nombre de la empresa y una lista de palabras clave.\n"
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """Crea el slogan de la empresa {empresa}.
para ello te han entregado esta lista de palabras clave {word_list}.
El slogan debe ser corto, creativo, pegajoso y reflejar la misión de la empresa.
Entrega en una linea el slogan y luego una breve explicacion del mismo.""",
    input_variables=["empresa", "word_list"],
)

Aqui podemos ver como se agregaron las variables que buscabamos en el user prompt

In [41]:
user_prompt.format(empresa=empresa, word_list=word_list)

HumanMessage(content="Crea el slogan de la empresa Tech Innovators.\npara ello te han entregado esta lista de palabras clave ['Innovate', 'Future', 'Tech', 'Solutions', 'NextGen'].\nEl slogan debe ser corto, creativo, pegajoso y reflejar la misión de la empresa.\nEntrega en una linea el slogan y luego una breve explicacion del mismo.", additional_kwargs={}, response_metadata={})

Ahora que tenemos el system y user prompts, podemos juntarlos en nuestro chat prompt completo usando ChatPromptTemplate:

In [42]:
from langchain.prompts import ChatPromptTemplate
myprompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

Por defecto, el **ChatPromptTemplate** leerá las variables de input de cada template entregado y permitirá a la vez usar dichas variables.

In [43]:
print(myprompt.format(empresa=empresa, word_list=word_list))

System: Eres un asistente creativo que genera slogans de empresas basados en el nombre de la empresa y una lista de palabras clave.

Human: Crea el slogan de la empresa Tech Innovators.
para ello te han entregado esta lista de palabras clave ['Innovate', 'Future', 'Tech', 'Solutions', 'NextGen'].
El slogan debe ser corto, creativo, pegajoso y reflejar la misión de la empresa.
Entrega en una linea el slogan y luego una breve explicacion del mismo.


ChatPromptTemplate también establece cada mensaje individual acorde a su rol. e.g.: System, Human, o AI.

Podemos encadenar nuestra plantilla myprompt y el objeto llm que definimos anteriormente para crear una cadena LLM simple.
Esta cadena realizará los pasos: formato del prompt > generación de llm > obtener la salida.

Usaremos el *LangChain Expression Language* (LCEL) para construir nuestra cadena.
Esta sintaxis puede parecer un poco extraña, pero la explicaremos en detalle más adelante en el curso.

Por ahora, solo necesitamos saber que definimos nuestras entradas con el primer segmento del diccionario
(es decir, {"empresa": lambda x: x["empresa"]}) y luego usamos el operador de pipe o palote (|) para indicar que la salida de la izquierda de la barra vertical se introducirá en la entrada de la derecha.

In [44]:
chain_one = (
    {"empresa": lambda x: x["empresa"], "word_list": lambda x: x["word_list"]}
    | myprompt
    | model
    | {"slogan": lambda x: x}
)

Esta es nuestra primera *Cadena de invocacion*

In [45]:
Slogan = chain_one.invoke({"empresa": empresa, "word_list": word_list})
Slogan

{'slogan': '"Soluciones para un futuro mejor"\n\nEste slogan busca capturar la esencia de la empresa Tech Innovators, enfocándose en la idea de brindar soluciones innovadoras que ayuden a impulsar el cambio positivo hacia el futuro. La palabra "nextGen" sugiere un enfoque en generaciones futuras y tecnología avanzada, lo cual se refleja en la estructura del slogan.'}

Refinando el Output

Ahora nos podemos apoyar de pydantic para crear un objeto que nos ayude a describir como queremos que sea el formato de salida.

In [46]:
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama

class SloganOutput(BaseModel):
    slogan: str = Field(description="El slogan creativo de la empresa.")
    explanation: str = Field(description="Una breve explicación del slogan.")
    feedback: str = Field(description="Recomendacion sobre otras palabras a usar.")

# Nueva objeto de model con output estructurado
structured_model = ChatOllama(model="llama3.2").with_structured_output(SloganOutput)

Ahora ponemos todo en conjunto, la nueva instancia con salida estructurada de pydantic, asi como los prompt templates

In [28]:
chain_three = (
    {"empresa": lambda x: x["empresa"], "word_list": lambda x: x["word_list"]}
    | myprompt
    | structured_model
    | {
        "slogan": lambda x: x.slogan,
        "explanation": lambda x: x.explanation,
        "feedback": lambda x: x.feedback
    }
)

In [29]:
output = chain_three.invoke({"empresa": empresa, "word_list": word_list})
output

{'slogan': 'Invierte en el futuro',
 'explanation': 'Este slogan destaca la capacidad de Tech Innovators para proporcionar soluciones tecnológicas que impulsan el progreso hacia un futuro más innovador, reflejando su compromiso con la innovación y la emoción por transformar el mundo.',
 'feedback': 'El objetivo del nombre es atraer a clientes innovadores que buscan tecnología de vanguardia para mejorar sus procesos internos y acceder a nuevas soluciones que les permitan mantenerse competitivos en un mercado cada vez más dinámico. La emoción de innovar puede ayudar a una empresa Tech Innovators a destacarse en el mercado. En este caso, la frase se ha utilizado al máximo, manteniendo todos los elementos claves y transmitiendo un mensaje claro.'}